# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
import json

In [2]:
import json

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# File to Load (Remember to Change These)
weather = "output_data/WeatherData.csv"

# Read School and Student Data File and store into Pandas DataFrames
df_weather = pd.read_csv(weather)


In [5]:
df_weather
df_weather = df_weather.rename(columns={'Latitude_x': 'Latitude'})
df_weather = df_weather.drop(columns=['Latitude_y'])
df_weather.head()

,CityName,Latitude,Temp,Humidity,Cloudiness,WindSpeed,Country,Long,NorthOrSouth
0,Kruisfontein,-34.00,71.01,70,15,3.00,ZA,24.73,South
1,Lavrentiya,65.58,32.38,94,94,4.65,RU,-171.00,North
2,Marawi,8.00,75.99,92,100,5.01,PH,124.29,North
3,East Massapequa,40.67,64.40,82,90,9.17,US,-73.44,North
4,New Norfolk,-42.78,42.80,60,40,18.34,AU,147.06,South


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
 # Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [12]:
# Store 'Lat' and 'Lng' into  locations 
locations = df_weather[["Latitude", "Long"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
df_weather = df_weather.dropna()
humi = df_weather["Humidity"].astype(float)

In [13]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humi, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
df_weather

,CityName,Latitude,Temp,Humidity,Cloudiness,WindSpeed,Country,Long,NorthOrSouth
0,Kruisfontein,-34.00,71.01,70,15,3.00,ZA,24.73,South
1,Lavrentiya,65.58,32.38,94,94,4.65,RU,-171.00,North
2,Marawi,8.00,75.99,92,100,5.01,PH,124.29,North
3,East Massapequa,40.67,64.40,82,90,9.17,US,-73.44,North
4,New Norfolk,-42.78,42.80,60,40,18.34,AU,147.06,South
...,...,...,...,...,...,...,...,...,...
561,Road Town,18.42,87.80,70,40,9.17,VG,-64.62,North
562,Safranbolu,41.25,67.30,37,0,1.86,TR,32.69,North
563,Martapura,-3.42,78.80,94,75,6.93,ID,114.85,South
564,Mahanoro,-19.90,72.16,75,6,3.27,MG,48.80,South


In [14]:
df_weathercriteria = df_weather.loc[(df_weather['Cloudiness']==0) & 
                                    (df_weather['WindSpeed'] < 10.0)&
                                    (df_weather['Temp'] <80.0) &
                                    (df_weather['Temp'] > 70.0) ]


In [15]:
df_weathercriteria

,CityName,Latitude,Temp,Humidity,Cloudiness,WindSpeed,Country,Long,NorthOrSouth
55,Rāhon,31.05,71.73,23,0,4.00,IN,76.12,North
203,Cape Town,-33.93,71.01,64,0,9.17,ZA,18.42,South
233,East London,-33.02,75.20,60,0,5.82,ZA,27.91,South
270,Giddarbāha,30.20,72.66,20,0,3.49,IN,74.67,North
326,Marrakesh,31.63,78.80,31,0,5.82,MA,-8.01,North
359,Jalu,29.03,73.89,37,0,9.19,LY,21.55,North
416,Esquipulas,14.57,78.80,83,0,2.24,GT,-89.35,North
425,Tūkrah,32.53,71.42,62,0,9.84,LY,20.58,North
441,Ibrā’,22.69,71.65,32,0,5.01,OM,58.53,North
545,Nkhotakota,-12.93,79.65,57,0,4.47,MW,34.30,South


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
narrowed_city_df = df_weathercriteria

In [17]:
narrowed_city_df["Hotel Name"] = ""

<ipython-input-17-75e371f1261c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  narrowed_city_df["Hotel Name"] = ""


In [18]:
narrowed_city_df

,CityName,Latitude,Temp,Humidity,Cloudiness,WindSpeed,Country,Long,NorthOrSouth,Hotel Name
55,Rāhon,31.05,71.73,23,0,4.00,IN,76.12,North,
203,Cape Town,-33.93,71.01,64,0,9.17,ZA,18.42,South,
233,East London,-33.02,75.20,60,0,5.82,ZA,27.91,South,
270,Giddarbāha,30.20,72.66,20,0,3.49,IN,74.67,North,
326,Marrakesh,31.63,78.80,31,0,5.82,MA,-8.01,North,
359,Jalu,29.03,73.89,37,0,9.19,LY,21.55,North,
416,Esquipulas,14.57,78.80,83,0,2.24,GT,-89.35,North,
425,Tūkrah,32.53,71.42,62,0,9.84,LY,20.58,North,
441,Ibrā’,22.69,71.65,32,0,5.01,OM,58.53,North,
545,Nkhotakota,-12.93,79.65,57,0,4.47,MW,34.30,South,


In [22]:
#TEST

# params dictionary to update each iteration
#params2 = {
 #   "radius": 5000,
 #   "types": "lodging",
 #   "location": "31.05,76.12",
 #   "key": g_key
#}

#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#testrequest = requests.get(base_url, params= params2)
#print(testrequest.json())
#print(json.dumps(testrequest.json(),indent=2))
#testrequest.json()["results"][0]["name"]
#hotelname = testrequest.json()["results"][0]["name"]

In [23]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "location": "location",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in narrowed_city_df.iterrows():
    lat = row["Latitude"]
    lng = row["Long"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}" #<=> "31.05,76.12"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    #print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        narrowed_city_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\zarre\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [24]:
narrowed_city_df

,CityName,Latitude,Temp,Humidity,Cloudiness,WindSpeed,Country,Long,NorthOrSouth,Hotel Name
55,Rāhon,31.05,71.73,23,0,4.00,IN,76.12,North,Bajwa home 🏠
203,Cape Town,-33.93,71.01,64,0,9.17,ZA,18.42,South,Southern Sun Waterfront Cape Town
233,East London,-33.02,75.20,60,0,5.82,ZA,27.91,South,Tu Casa
270,Giddarbāha,30.20,72.66,20,0,3.49,IN,74.67,North,Grover'S
326,Marrakesh,31.63,78.80,31,0,5.82,MA,-8.01,North,Les Jardins de La Koutoubia
359,Jalu,29.03,73.89,37,0,9.19,LY,21.55,North,Jalu Hotel
416,Esquipulas,14.57,78.80,83,0,2.24,GT,-89.35,North,Grand Caporal Hotel
425,Tūkrah,32.53,71.42,62,0,9.84,LY,20.58,North,استراحة لاقروتا
441,Ibrā’,22.69,71.65,32,0,5.01,OM,58.53,North,مزرعة راحة النفوس المنزفة
545,Nkhotakota,-12.93,79.65,57,0,4.47,MW,34.30,South,Sitima Inn


In [25]:
# Store 'Lat' and 'Lng' into  locations 
locations = narrowed_city_df[["Latitude", "Long"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
#df_weather = df_weather.dropna()
humi2 = narrowed_city_df["Humidity"].astype(float)

In [26]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humi2, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
narrowed_city_df

,CityName,Latitude,Temp,Humidity,Cloudiness,WindSpeed,Country,Long,NorthOrSouth,Hotel Name
55,Rāhon,31.05,71.73,23,0,4.00,IN,76.12,North,Bajwa home 🏠
203,Cape Town,-33.93,71.01,64,0,9.17,ZA,18.42,South,Southern Sun Waterfront Cape Town
233,East London,-33.02,75.20,60,0,5.82,ZA,27.91,South,Tu Casa
270,Giddarbāha,30.20,72.66,20,0,3.49,IN,74.67,North,Grover'S
326,Marrakesh,31.63,78.80,31,0,5.82,MA,-8.01,North,Les Jardins de La Koutoubia
359,Jalu,29.03,73.89,37,0,9.19,LY,21.55,North,Jalu Hotel
416,Esquipulas,14.57,78.80,83,0,2.24,GT,-89.35,North,Grand Caporal Hotel
425,Tūkrah,32.53,71.42,62,0,9.84,LY,20.58,North,استراحة لاقروتا
441,Ibrā’,22.69,71.65,32,0,5.01,OM,58.53,North,مزرعة راحة النفوس المنزفة
545,Nkhotakota,-12.93,79.65,57,0,4.47,MW,34.30,South,Sitima Inn


In [32]:
hotel_df = narrowed_city_df[["Hotel Name","CityName", "Country","Latitude","Long"]]
#hotel_df.dtypes

In [33]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{CityName}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Latitude", "Long"]]


In [36]:
# Add marker layer ontop of heat map


marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
#fig

fig.add_layer(heat_layer)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))